# ACCEL IMPLEMENTATION

In [4]:
# DANGER: ACCEL does not use a teacher, but a single student that learns from levels selected by a curator, in this impmenetation the curator is a sampler that gives te studemt with a probabilistic manner, the highest regret-based score level, but instaed of discarding the levels during training, it makes small edits to the most diffucult ones, gibing the change to rely on already good but hard leves without the need of a teacher.
# TODO: the start_pos and goal_pos of edited levels are uncorrectly marked, if I mutate the level no

In [5]:
import torch

import numpy as np
import gymnasium as gym

from gymnasium.spaces import Box

from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Goal, Wall
from minigrid.minigrid_env import MiniGridEnv, Grid

from minigrid.wrappers import ImgObsWrapper

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

import torch.nn as nn

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# ====================================================
# 1. Custom MiniGrid Environment that returns only the image
#    for SB3's PPO (which expects a Box space).
# ====================================================
class MyCustomGrid(MiniGridEnv):
    """
    Simple MiniGrid environment that places random wall tiles
    according to a config dict, returning only the 'image' observation.
    """

    def __init__(self, config=None, **kwargs):
        if config is None:
            config = {}
        self.config = config

        # Extract parameters from config
        self.width = config.get("width")
        self.height = config.get("height")
        self.num_blocks = config.get("num_blocks")
        self.custom_seed = config.get("seed_val")
        
        
        # Create a random number generator with the custom seed
        self.rng = np.random.default_rng(seed=self.custom_seed)

        grid_size = max(self.width, self.height)

        mission_space = MissionSpace(mission_func=lambda: "get to the green goal square")

        super().__init__(
            grid_size=grid_size,
            max_steps=self.width * self.height * 2, # max_steps is typically 2x the grid size
            see_through_walls=False,
            agent_view_size=5,                      # Size of the agent's view square
            mission_space=mission_space,
            **kwargs
        )

        # Manually define our observation_space as a single Box (the image).
        # By default, MiniGrid's image shape is (view_size, view_size, 3) if using partial obs,
        # or (height, width, 3) if using full-grid observation. We'll do full-grid here:
        # We'll define (self.height, self.width, 3) as the shape.
        # In practice, "image" shape can vary if partial observations are used.
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(self.agent_view_size, self.agent_view_size, 3),
            dtype=np.uint8
        )

    def _gen_grid(self, width, height):
        """
        Generate the grid layout for a new episode.
        We use self.width and self.height from config, even though the underlying
        MiniGrid environment might use grid_size for some of its operations.
        """    
        
        # Create an empty grid of the "true" width x height from config
        self.grid = Grid(self.width, self.height)
        # Surround the grid with walls
        self.grid.wall_rect(0, 0, self.width, self.height)
        
        # Place random walls inside using the custom seed. Only place a wall if the cell is empty.
        for _ in range(self.num_blocks):
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None: #and (c, r) != self.config["start_pos"] and (c, r) != self.config["goal_pos"]:
                self.put_obj(Wall(), c, r)
        
        # Place the goal object in a random position not occupied by any wall
        """if self.config["goal_pos"] is None:"""
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None: # and (c, r) != self.config["start_pos"]:
                self.put_obj(Goal(), c, r)
                self.config["goal_pos"] = (c, r)
                break
        """elif self.config["goal_pos"] is not None:
            c, r = self.config["goal_pos"]
            self.put_obj(Goal(), c, r)"""

        # Place the agent in a random position not occupied by any wall and not on the goal
        
        """if self.config["start_pos"] is None:"""
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None: # and (c, r) != self.config["goal_pos"]:
                self.place_agent(top=(c, r), rand_dir=True)
                self.config["start_pos"] = (c, r)
                break  
        """elif self.config["start_pos"] is not None:
            c, r = self.config["start_pos"]
            self.place_agent(top=(c, r), rand_dir=True)"""
     

    def reset(self, **kwargs):
        """
        Override reset to ensure we only return the 'image' array
        instead of a dict with 'image' and 'mission'.
        """
        obs, info = super().reset(**kwargs)
        obs = self._convert_obs(obs)
        return obs, info

    def step(self, action):
        """
        Same for step: override to convert the dict observation into an image only.
        """
        obs, reward, done, truncated, info = super().step(action)
        obs = self._convert_obs(obs)
        return obs, reward, done, truncated, info

    def _convert_obs(self, original_obs):
        """
        original_obs is typically {'image':..., 'mission':...}.
        We'll just return original_obs['image'] to get a Box(low=0,high=255) shape.
        """
        #print(original_obs["image"].shape)
        return original_obs["image"]
        #return np.transpose(original_obs["image"], (2, 0, 1))



def random_config(grid_size, num_blocks=None):
    max_blocks = int(((grid_size - 1) * (grid_size - 1)) / 2)
    
    if num_blocks is None:
        num_blocks = np.random.randint(1, max_blocks)
    else:
        num_blocks = min(num_blocks, max_blocks)
    
    return {
        "width": grid_size,
        "height": grid_size,
        "num_blocks": num_blocks,
        "start_pos": None,
        "goal_pos": None,
        "edited": False,
        "seed_val": np.random.randint(0, 999999),
    }
    
# Modify an existing configuration, adding randomness.
def edit_config(old_config):
    max_blocks = int(((old_config["width"] - 1) * (old_config["height"] - 1)) / 2)
    
    new_config = dict(old_config)
    
    # Randomly change the number of blocks
    new_number_blocks = old_config["num_blocks"] + np.random.choice([1])
    
    # Ensure the number of blocks is within bounds
    new_config["num_blocks"] = max(1, min(new_number_blocks, max_blocks))    
    
    # Mark the config as edited
    new_config["edited"] = True
    
    return new_config



# ====================================================
# 2. Simple “level buffer” 
# ====================================================
# class to memorize generated levels and score
class LevelBuffer: 
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []  # will store (config_dict, score)

    def add(self, config, score):
        self.data.append((config, score))
        if len(self.data) > self.max_size:
            self.data.sort(key=lambda x: x[1], reverse=True)
            self.data = self.data[: self.max_size]
            #it memorize only the highest score for each level

    def sample_config(self): 
        # Samples a level from the buffer, weighting the probabilities 
        # based on the scores.
        if len(self.data) == 0:
            return None
        scores = [item[1] for item in self.data]
        total = sum(scores)
        if total <= 1e-9:
            # fallback to uniform
            idx = np.random.randint(len(self.data))
            return self.data[idx][0]
        probs = [s / total for s in scores]
        idx = np.random.choice(len(self.data), p=probs)
        return self.data[idx][0]

# ====================================================
# 3. Regret Calculation with Generalized Advantage Estimation (GAE)
# ====================================================

# Calculate regret using Generalized Advantage Estimation (GAE) with Stable-Baselines3's PPO model.
# PLR approximates regret using a score function such as the positive value loss.
def calculate_regret_gae(env, model, max_steps, gamma, lam):
    """
    Calculate regret using Generalized Advantage Estimation (GAE)
    with Stable-Baselines3's PPO model.
    """
    obs, _ = env.reset()
    regrets = []
    rewards = []
    dones = []
    values = []

    for t in range(max_steps):
        # Add batch dimension to the observation tensor
        #print("I am here")
        # Transform obs to pytorch tensor
        obs_transposed = np.transpose(obs, (2, 0, 1))
        obs = np.expand_dims(obs_transposed, axis=0)
        obs_tensor = torch.as_tensor(obs).float().to(device)
        
        # Use the model's policy to get the value and action.
        # For actions, model.predict handles single observations well.
        action, _ = model.predict(obs, deterministic=True)
        
        #print("Action is:", action)
        
        # Compute the value from the policy.
        value_t = model.policy.predict_values(obs_tensor).item()
        values.append(value_t)
        
        #print("Value is:", value_t) # THE CODE STOPS HERE
        
        # Perform the step in the environment
        obs, reward, done, truncated, _ = env.step(action)
        rewards.append(reward)
        dones.append(done)

        if done or truncated:
            break

    #print("I am here, outside the loop")
    # Add value of the terminal state (0 if done/truncated)
    if done or truncated:
        terminal_value = 0.0
    else:
        terminal_obs_tensor = torch.as_tensor(obs).float().unsqueeze(0).to(device)
        terminal_value = model.policy.predict_values(terminal_obs_tensor).item()
    values.append(terminal_value)

    # Compute TD-errors and GAE-like regret score
    for t in range(len(rewards)):
        delta_t = rewards[t] + gamma * values[t + 1] * (1 - dones[t]) - values[t]
        discounted_error = (gamma * lam) ** t * delta_t
        regrets.append(max(0, discounted_error))

    # Return the maximum positive regret score (or 0 if empty)
    return max(regrets) if regrets else 0.0

# ====================================================
# 3. Regret Calculation with Generalized Advantage Estimation (GAE) in Parallel
# ====================================================

def calculate_regret_gae_parallel(
    env_config, 
    model, 
    max_steps=1000, 
    gamma=0.99, 
    lam=0.95, 
    n_envs=4
):
    """
    Roll out n_envs copies of MyCustomGrid(env_config) in parallel,
    compute GAE-based 'regret' for each environment, and return the max.
    """

    # Create vectorized env with n_envs copies
    vec_env = create_vectorized_env(env_config, n_envs=n_envs)
    obs_array = vec_env.reset()  # shape: (n_envs, height, width, 3)

    # For each environment, we will store transitions to later compute GAE
    # We'll keep them in lists, one per environment.
    # Alternatively, we can store them in big arrays (n_envs, max_steps), etc.
    rewards_list = [[] for _ in range(n_envs)]
    values_list = [[] for _ in range(n_envs)]
    dones_list = [[] for _ in range(n_envs)]

    for t in range(max_steps):
        # Model’s predict can handle multiple obs in a single forward pass
        actions, _ = model.predict(obs_array, deterministic=True)
        
        # Also compute the values in one batch
        # Convert obs_array to torch tensor
        obs_tensor = torch.as_tensor(np.transpose(obs_array, (0, 3, 1, 2))).float().to(device)
        with torch.no_grad():
            # shape: (n_envs, 1)
            value_t = model.policy.predict_values(obs_tensor).cpu().numpy().flatten()

        # Step all envs in parallel
        next_obs_array, rewards, dones, truncs = vec_env.step(actions)

        # Store the results
        for i in range(n_envs):
            rewards_list[i].append(rewards[i])
            values_list[i].append(value_t[i])
            dones_list[i].append(bool(dones[i]) or bool(truncs[i]))

        obs_array = next_obs_array

        # If all envs are done or truncated, we can break early
        if all(dones) or all(truncs):
            break

    # We also need the terminal value for each env
    # (0 if done, otherwise model's value at final obs)
    obs_tensor = torch.as_tensor(np.transpose(obs_array, (0, 3, 1, 2))).float().to(device)
    with torch.no_grad():
        final_values = model.policy.predict_values(obs_tensor).cpu().numpy().flatten()

    # Now, compute GAE-based "regret" for each of the n_envs
    regrets = []
    for i in range(n_envs):
        # If the env ended with done or truncated, terminal value = 0
        if dones_list[i][-1]:
            values_list[i].append(0.0)
        else:
            values_list[i].append(final_values[i])

        # Compute delta_t and approximate GAE-like metric
        env_rewards = rewards_list[i]
        env_values = values_list[i]
        env_dones = dones_list[i]

        env_regrets = []
        for t in range(len(env_rewards)):
            #print("env_rewards[t]:", env_rewards[t])
            #print("gamma", gamma)
            #print("env_values[t + 1]:", env_values[t + 1])
            #print("env_dones[t]:", env_dones[t].get('TimeLimit.truncated', False))
            #print("env_values[t]:", env_values[t])
            delta_t = env_rewards[t] + gamma * env_values[t + 1] * (1 - env_dones[t]) - env_values[t]
            # accumulate discounted error
            discounted_error = (gamma * lam) ** t * delta_t
            env_regrets.append(max(0, discounted_error))

        # The environment's "regret" is the max of its positive GAE deltas
        regrets.append(max(env_regrets) if env_regrets else 0.0)

    # Return the maximum regret across the parallel envs
    return max(regrets) if regrets else 0.0

# ====================================================
# 4. Custom Features Extractor for MiniGrid
# ====================================================

class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 512, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        #print("The observation space given to the features extractor is:", observation_space)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            n_flatten = self.cnn(torch.as_tensor(observation_space.sample()[None]).float()).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))


policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)

def initialize_ppo(env, learning_rate=1e-4):
    return PPO(
        "CnnPolicy",                    # Convolutional neural network policy (For GPU)
        env,                            # environment to learn from
        verbose=0,                      # Display training output
        n_steps=256,                    # Number of steps to run for each environment per update
        batch_size=256,                  # Minibatch size for each gradient update
        learning_rate=learning_rate,
        policy_kwargs=policy_kwargs,    # Custom policy arguments
        device=device                   # Use GPU if available
    )

def print_level_from_config(config):
    env = MyCustomGrid(config, render_mode='rgb_array')
    env.reset()
    full_level_image = env.render()  # This should return an RGB image of the full grid

    plt.figure(figsize=(4, 4))
    plt.imshow(full_level_image)
    plt.title("Level Configuration: " + str(config))
    plt.axis("off")
    plt.show()
    
# Use vectorized environment
def create_vectorized_env(config, n_envs=4):
    """
    Create a vectorized environment with n parallel environments.
    """
    env_fns = [lambda: MyCustomGrid(config) for _ in range(n_envs)]
    return make_vec_env(lambda: MyCustomGrid(config), n_envs=n_envs)


import time

def main_accel(total_iterations, replay_prob, train_steps, level_buffer_size,
                    initial_fill_size, grid_size, n_envs, edit_levels, regret_threshold,
                    easy_start):
    # Create a level buffer
    level_buffer = LevelBuffer(max_size=level_buffer_size)
    iteration_regrets = []
    
    # Use a random config to create a vectorized environment
    dummy_config = random_config(grid_size)
    vectorized_env = create_vectorized_env(dummy_config, n_envs=n_envs)
    print("Vectorized environment created.")
  
    # Initialize PPO with vectorized environment
    print("Initializing student model PPO...")
    student_model = initialize_ppo(vectorized_env)

    # Populate buffer with initial levels
    print(f"Populating buffer with {initial_fill_size} initial levels with regret > {regret_threshold}...")
    
    start_time = time.time()
    while len(level_buffer.data) < initial_fill_size:
        if easy_start:
            cfg = random_config(grid_size, num_blocks=2)
        else:
            cfg = random_config(grid_size)
            
        regret = calculate_regret_gae_parallel(cfg, student_model, max_steps=1000, gamma=0.99, lam=0.95, n_envs=n_envs)
        #regret = calculate_regret_gae(MyCustomGrid(cfg), student_model, max_steps=500, gamma=0.99, lam=0.95)
        
        #print(f"Regret for initial level: {regret}, {len(level_buffer.data)}")

        # Skip levels with low regret
        if regret < regret_threshold:
            continue

        level_buffer.add(cfg, regret)
    end_time = time.time()
    
    print(f"Initial buffer fill took {end_time - start_time:.2f} seconds.")
    

    # Main ACCEL loop
    iteration, skipped = 0, 0
    print("\nMain training loop...")
    while iteration < total_iterations + skipped:
        print(f"\n=== ITERATION {iteration + 1}/{total_iterations + skipped} SKIPPED: {skipped} ===")
        
        iteration += 1
        
        # Decide whether to replay or generate a new level
        use_replay = np.random.rand() < replay_prob

        if not use_replay or len(level_buffer.data) == 0:
            # Create a new random level
            cfg = random_config(grid_size)
            print("Generated new random level:", cfg)
        else:
            # Sample a level from the buffer
            cfg = level_buffer.sample_config()
            print("Sampled level from buffer:", cfg)
        
                
        # Update the vectorized environment with the selected config and train the model
        vectorized_env = create_vectorized_env(cfg, n_envs=n_envs)
        student_model.set_env(vectorized_env)
        student_model.learn(total_timesteps=train_steps)



        if not use_replay or not edit_levels:
            regret = calculate_regret_gae_parallel(cfg, student_model, max_steps=1000, gamma=0.99, lam=0.95, n_envs=n_envs)
        else:
            # Edit the level and calculate regret
            cfg = edit_config(cfg)
            print("Edited level to:", cfg)
            regret = calculate_regret_gae_parallel(cfg, student_model, max_steps=1000, gamma=0.99, lam=0.95, n_envs=n_envs)
        
        
        if regret <= regret_threshold:
            print(f"Regret for current level is {regret:.5f} <= threshold {regret_threshold}. Skipping...")
            skipped += 1
            continue

        
        print(f"Regret for current level: {regret}")
        level_buffer.add(cfg, regret)
        iteration_regrets.append(regret)
        
    

    # Plot and display the progress
    plt.figure(figsize=(8, 4))
    plt.plot(iteration_regrets, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Regret")
    plt.title("Regret Progress during ACCEL Training")
    plt.grid(True)
    plt.show()
    
    print("\nDone. Final buffer size:", len(level_buffer.data))
    print("Top-5 hardest levels (config, regret):")
    level_buffer.data.sort(key=lambda x: x[1], reverse=True)
    for i, (cfg, sc) in enumerate(level_buffer.data[:5]):
        print(f"{i + 1}. regret={sc:.5f}, config={cfg}")
        #print_level_from_config(cfg)
        
    print("Top-5 easiest levels (config, regret):")
    level_buffer.data.sort(key=lambda x: x[1])
    for i, (cfg, sc) in enumerate(level_buffer.data[:5]):
        print(f"{i + 1}. regret={sc:.5f}, config={cfg}")
        #print_level_from_config(cfg)
    
    return student_model

cuda


# TRAIN

In [6]:
%%prun -s cumulative


# set all possible seeds    
torch.manual_seed(42)
np.random.seed(42)




config = {
        "grid_size": 8,
        
        "total_iterations": 15,
        "train_steps": 500,

        "replay_prob": 0.7,           # Probability of replaying a level and editing it vs. generating a new one
        "level_buffer_size": 128,     # Maximum number of levels to store in the buffer
        "initial_fill_size": 64,      # Number of levels to pre-fill the buffer with
        "regret_threshold": 0.0,      # Minimum regret threshold to consider a level for the buffer
        
        "n_envs": 3,                  # Number of parallel environments to use for training
        
        "edit_levels": True,          # Whether to edit levels during training i.e. ACCEL or PLR
        "easy_start": True            # Whether to fill the buffer with easy levels first i.e. minimum number of blocks
    }


'''
config["edit_levels"] = False
config["easy_start"] = False
print(f"Running PLR with config: {config}")
model_plr = main_accel(**config)

print("\n\n============================================\n\n")

config["edit_levels"] = True
config["easy_start"] = False
print(f"Running ACCEL with config: {config}")
model_accel = main_accel(**config)

print("\n\n============================================\n\n")
'''

config["edit_levels"] = True
config["easy_start"] = True
print(f"Running ACCEL with easy start with config: {config}")
model_accel_easy = main_accel(**config)

Running ACCEL with easy start with config: {'grid_size': 8, 'total_iterations': 15, 'train_steps': 500, 'replay_prob': 0.7, 'level_buffer_size': 128, 'initial_fill_size': 64, 'regret_threshold': 0.0, 'n_envs': 3, 'edit_levels': True, 'easy_start': True}
Vectorized environment created.
Initializing student model PPO...
Populating buffer with 64 initial levels with regret > 0.0...
Initial buffer fill took 0.79 seconds.

Main training loop...

=== ITERATION 1/15 SKIPPED: 0 ===
Generated new random level: {'width': 8, 'height': 8, 'num_blocks': 1, 'start_pos': None, 'goal_pos': None, 'edited': False, 'seed_val': 48984}
Regret for current level is 0.00000 <= threshold 0.0. Skipping...

=== ITERATION 2/16 SKIPPED: 1 ===
Sampled level from buffer: {'width': 8, 'height': 8, 'num_blocks': 2, 'start_pos': (2, 6), 'goal_pos': (6, 5), 'edited': False, 'seed_val': 256840}
Edited level to: {'width': 8, 'height': 8, 'num_blocks': 3, 'start_pos': (6, 2), 'goal_pos': (1, 6), 'edited': True, 'seed_val':

KeyboardInterrupt: 

# EVALUATION

In [ ]:
models = {'PLR': model_plr, 'ACCEL': model_accel, 'ACCEL-EasyStart': model_accel_easy}
models = {'ACCEL-EasyStart': model_accel_easy}
difficulties = 3
# Generate n levels with increasing complexity, for each level generate 10 configs
levels = []
for i in range(difficulties):
    level = []
    for _ in range(10):
        cfg = random_config(config["grid_size"], num_blocks=i*i*3)
        #print_level_from_config(cfg)
        level.append(cfg)
    levels.append(level)

# Evaluate the model on the generated levels
results = {}
for model_name, model in models.items():
    results[model_name] = []
    for i, level in enumerate(levels):
        print(f"Evaluating level {i + 1} with {4 + i*i} blocks for model {model_name}...")
        r = []
        for j, cfg in enumerate(level):
            # Create vectorized environment
            env = create_vectorized_env(cfg, n_envs=8)
            mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=20, deterministic=True)
            r.append(mean_reward)
        results[model_name].append(r)
    print()
    
# Print mean rewards for each level
for model_name in models.keys():
    print(f"Model: {model_name}")
    for i, level in enumerate(levels):
        print(f"Level {i + 1} - Complexity {4 + i*i}: {np.mean(results[model_name][i]):.2f}")
    print()

# Boxplot of results, a plot for each level complexity comparing models
plt.figure(figsize=(12, 6))
for i, level in enumerate(levels):
    plt.subplot(1, difficulties, i + 1)
    plt.boxplot([results[model_name][i] for model_name in models.keys()])
    plt.xticks([1, 2, 3], [model_name for model_name in models.keys()])
    plt.title(f"Level {i + 1} - Complexity {4 + i*3}")
    plt.ylabel("Mean Reward")
plt.tight_layout()
plt.show()

In [ ]:
def test_model(model, config):
    env = MyCustomGrid(config, render_mode='human')
    obs, _ = env.reset()
    terminated = False
    truncated = False
    total_reward = 0
    i = 0
    while not terminated or not truncated:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        total_reward += reward
        i += 1
        if i > 20:
            break
    return total_reward

# Test the models on a few levels
for i in range(30):
    test_model(model_accel_easy, random_config(8))

# Level Editor Tests

In [ ]:
# Generate a random level and visualize it
random_cnf = random_config(8)
print_level_from_config(random_cnf)
print("random_cnf:", random_cnf)

# Edit the random level and visualize it
edited_config = edit_config(random_cnf)
print_level_from_config(edited_config)

# Edit the random level and visualize it
edited_config = edit_config(random_cnf)
print_level_from_config(edited_config)

# Edit the random level and visualize it
edited_config = edit_config(random_cnf)
print_level_from_config(edited_config)

# Maze generator

In [ ]:
import random

def generate_maze(width, height):
    maze = [[1 for _ in range(width)] for _ in range(height)]  # 1 for walls
    stack = []
    directions = [(0, 2), (0, -2), (2, 0), (-2, 0)]

    def is_valid(x, y):
        return 0 < x < height - 1 and 0 < y < width - 1 and maze[x][y] == 1

    def carve(x, y):
        maze[x][y] = 0
        random.shuffle(directions)
        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid(nx, ny):
                maze[x + dx // 2][y + dy // 2] = 0
                carve(nx, ny)

    carve(1, 1)  # Start at (1, 1)

    return maze

# Display the maze
def print_maze(maze):
    for row in maze:
        print("".join("█" if cell == 1 else " " for cell in row))

maze = generate_maze(21, 21)
print_maze(maze)


In [ ]:
class MyCustomMaze(MiniGridEnv):
    """
    Simple MiniGrid environment that places random wall tiles
    according to a config dict, returning only the 'image' observation.
    """

    def __init__(self, config=None, **kwargs):
        if config is None:
            config = {}
        self.config = config

        # Extract parameters from config
        self.width = config.get("width")
        self.height = config.get("height")
        self.num_blocks = config.get("num_blocks")
        self.custom_seed = config.get("seed_val")
        
        
        # Create a random number generator with the custom seed
        self.rng = np.random.default_rng(seed=self.custom_seed)

        grid_size = max(self.width, self.height)

        mission_space = MissionSpace(mission_func=lambda: "get to the green goal square")

        super().__init__(
            grid_size=grid_size,
            max_steps=self.width * self.height * 2, # max_steps is typically 2x the grid size
            see_through_walls=False,
            agent_view_size=5,                      # Size of the agent's view square
            mission_space=mission_space,
            **kwargs
        )

        # Manually define our observation_space as a single Box (the image).
        # By default, MiniGrid's image shape is (view_size, view_size, 3) if using partial obs,
        # or (height, width, 3) if using full-grid observation. We'll do full-grid here:
        # We'll define (self.height, self.width, 3) as the shape.
        # In practice, "image" shape can vary if partial observations are used.
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(self.agent_view_size, self.agent_view_size, 3),
            dtype=np.uint8
        )

    
    def _gen_grid(self, width, height):
        """
        Generate the grid layout for a new episode using the DFS Maze Generation Algorithm.
        """
        # Create an empty grid of the "true" width x height from config
        self.grid = Grid(self.width, self.height)
        # Surround the grid with walls
        self.grid.wall_rect(0, 0, self.width, self.height)

        # Initialize the maze as walls
        maze = [[1 for _ in range(self.width)] for _ in range(self.height)]

        # Define directions for DFS
        directions = [(0, 2), (0, -2), (2, 0), (-2, 0)]

        def is_valid(x, y):
            """Check if a cell is valid for carving."""
            return 0 < x < self.height - 1 and 0 < y < self.width - 1 and maze[x][y] == 1

        def carve(x, y):
            """Carve passages in the maze using DFS."""
            maze[x][y] = 0  # Mark the cell as part of the maze
            self.grid.set(x, y, None)  # Clear the wall in the grid
            self.rng.shuffle(directions)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if is_valid(nx, ny):
                    # Remove the wall between cells
                    maze[x + dx // 2][y + dy // 2] = 0
                    self.grid.set(x + dx // 2, y + dy // 2, None)
                    carve(nx, ny)

        # Start carving from the top-left corner
        carve(1, 1)

        # Place the goal object in a random position not occupied by a wall
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.put_obj(Goal(), c, r)
                break

        # Place the agent in a random position not occupied by a wall and not on the goal
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.place_agent(top=(c, r), rand_dir=True)
                break
    
    
    def reset(self, **kwargs):
        """
        Override reset to ensure we only return the 'image' array
        instead of a dict with 'image' and 'mission'.
        """
        obs, info = super().reset(**kwargs)
        obs = self._convert_obs(obs)
        return obs, info

    def step(self, action):
        """
        Same for step: override to convert the dict observation into an image only.
        """
        obs, reward, done, truncated, info = super().step(action)
        obs = self._convert_obs(obs)
        return obs, reward, done, truncated, info

    def _convert_obs(self, original_obs):
        """
        original_obs is typically {'image':..., 'mission':...}.
        We'll just return original_obs['image'] to get a Box(low=0,high=255) shape.
        """
        return original_obs["image"]
        #return np.transpose(original_obs["image"], (2, 0, 1))



def print_maze_from_config(config):
    env = MyCustomMaze(config, render_mode='rgb_array')
    env.reset()
    full_level_image = env.render()  # This should return an RGB image of the full grid

    plt.figure(figsize=(4, 4))
    plt.imshow(full_level_image)
    plt.title("Maze Configuration: " + str(config))
    plt.axis("off")
    plt.show()
    

# Generate a random maze and visualize it
random_maze = random_config(6)
print_maze_from_config(random_maze)



# OLD CODE, NOT VECORIZED, REDUNDANT CODE

In [10]:
"""
# ====================================================
# 4. Main ACCEL Loop
# ====================================================

def main_accel_demo(total_iterations, replay_prob, train_steps, level_buffer_size,
                    initial_fill_size, grid_size):
    
    
    # Create a level buffer to store generated levels and their scores
    level_buffer = LevelBuffer(max_size=level_buffer_size)
    iteration_regrets = []
        
    #Create a dummy environment to initialize the model
    dummy_env = MyCustomGrid(random_config(grid_size))
    vectorized_env = create_vectorized_env(dummy_env, n_envs=4)

    # Initialize student model with PPO
    print("Initializing student model PPO...")
    student_model = initialize_ppo(dummy_env)

    skipped = 0

    # Populate buffer with initial levels
    print(f"Populating buffer with {initial_fill_size} initial levels with regret != 0...")
    for _ in range(initial_fill_size + skipped):
        cfg = random_config(grid_size)
        regret = calculate_regret_gae(MyCustomGrid(cfg), student_model, max_steps=1000, gamma=0.99, lam=0.95)
        
        # Skip levels with 0 regret
        if regret == 0:
            skipped += 1
            continue
        
        level_buffer.add(cfg, regret)
        
    
    print("\nDone. Number of skipped levels with zero regret:", skipped)
    
    skipped = 0
    iteration = 0
    # Main ACCEL loop
    print("\nMain ACCEL loop...")
    while iteration < total_iterations + skipped:
        print(f"\n=== ITERATION {iteration + 1}/{total_iterations} SKIPPED {skipped} ===")
        
        iteration += 1
        
        # Decide whether to use replay or generate a new level
        use_replay = np.random.rand() < replay_prob
        
        # Generates new random levels if you don't use replay
        if not use_replay or len(level_buffer.data) == 0:
            cfg = random_config(grid_size)
            regret = calculate_regret_gae(MyCustomGrid(cfg), student_model, max_steps=100, gamma=0.99, lam=0.95)
            
            if regret == 0:
                skipped += 1
                continue
            
            level_buffer.add(cfg, regret)
            print(f"  Sampled new config, regret={regret:.3f}")
        else:
            # Replays an existing layer, edits it, and evaluates the new layer
            old_cfg = level_buffer.sample_config()
            env = MyCustomGrid(old_cfg)
            
            student_model.set_env(env)
            student_model.learn(total_timesteps=train_steps)

            new_cfg = edit_config(old_cfg)
            regret = calculate_regret_gae(MyCustomGrid(new_cfg), student_model, max_steps=100, gamma=0.99, lam=0.95)
            
            if regret == 0:
                skipped += 1
                continue
            
            level_buffer.add(new_cfg, regret)
            print(f"  Replayed + mutated config, regret={regret:.3f}")
        
        iteration_regrets.append(regret)
    
    print("\nDone. Number of skipped levels with zero regret:", skipped)

    # Visualize progress of the regret over iterations.
    plt.figure(figsize=(8, 4))
    plt.plot(iteration_regrets, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Regret")
    plt.title("Regret Progress during ACCEL Training")
    plt.grid(True)
    plt.show()
    
    
    

if __name__ == "__main__":
    
    
    config = {
        "grid_size": 8,
        
        "total_iterations": 64,
        "train_steps": 1024,

        "replay_prob": 0.7,           # Probability of replaying a level and editing it vs. generating a new one
        "level_buffer_size": 128,     # Maximum number of levels to store in the buffer
        "initial_fill_size": 64,      # Number of levels to pre-fill the buffer with
        "regret_threshold": 0.0,      # Minimum regret threshold to consider a level for the buffer
        
        "n_envs": 8,                  # Number of parallel environments to use for training
        
        "edit_levels": True,          # Whether to edit levels during training i.e. ACCEL or PLR
        "easy_start": True            # Whether to fill the buffer with easy levels first i.e. minimum number of blocks
    }
    
    print("Running ACCEL with config:")
    print(config, "\n")
    
    main_accel(**config)

"""

In [ ]:
import torch
import numpy as np
import gymnasium as gym
from gymnasium.spaces import Box

from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Goal, Wall
from minigrid.minigrid_env import MiniGridEnv, Grid

from minigrid.wrappers import ImgObsWrapper

from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

import torch.nn as nn
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# ====================================================
# 1. Custom MiniGrid Environment that returns only the image
#    for SB3's PPO (which expects a Box space).
# ====================================================
class MyCustomGrid(MiniGridEnv):
    """
    Simple MiniGrid environment that places random wall tiles
    according to a config dict, returning only the 'image' observation.
    """

    def __init__(self, config=None, **kwargs):
        if config is None:
            config = {}
        self.config = config

        # Extract parameters from config
        self.width = config.get("width")
        self.height = config.get("height")
        self.num_blocks = config.get("num_blocks")
        self.custom_seed = config.get("seed_val")

        # Create a random number generator with the custom seed
        self.rng = np.random.default_rng(seed=self.custom_seed)

        grid_size = max(self.width, self.height)

        mission_space = MissionSpace(mission_func=lambda: "get to the green goal square")

        super().__init__(
            grid_size=grid_size,
            max_steps=self.width * self.height * 2, # typical 2x the grid size
            see_through_walls=False,
            agent_view_size=5,
            mission_space=mission_space,
            **kwargs
        )

        # Manually define our observation_space as a single Box (the image).
        self.observation_space = Box(
            low=0,
            high=255,
            shape=(self.agent_view_size, self.agent_view_size, 3),
            dtype=np.uint8
        )

    def _gen_grid(self, width, height):
        """
        Generate the grid layout for a new episode.
        """

        # Create an empty grid of the "true" width x height from config
        self.grid = Grid(self.width, self.height)
        # Surround the grid with walls
        self.grid.wall_rect(0, 0, self.width, self.height)

        # Place random walls inside using the custom seed. 
        for _ in range(self.num_blocks):
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.put_obj(Wall(), c, r)

        # Place the goal object in a random position not occupied by any wall
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.put_obj(Goal(), c, r)
                self.config["goal_pos"] = (c, r)
                break

        # Place the agent in a random position not occupied by any wall and not on the goal
        while True:
            r = self.rng.integers(1, self.height - 1)
            c = self.rng.integers(1, self.width - 1)
            if self.grid.get(c, r) is None:
                self.place_agent(top=(c, r), rand_dir=True)
                self.config["start_pos"] = (c, r)
                break

    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)
        obs = self._convert_obs(obs)
        return obs, info

    def step(self, action):
        obs, reward, done, truncated, info = super().step(action)
        obs = self._convert_obs(obs)
        return obs, reward, done, truncated, info

    def _convert_obs(self, original_obs):
        return original_obs["image"]


def random_config(grid_size, num_blocks=None):
    max_blocks = int(((grid_size - 1) * (grid_size - 1)) / 2)
    
    if num_blocks is None:
        num_blocks = np.random.randint(1, max_blocks)
    else:
        num_blocks = min(num_blocks, max_blocks)
    
    return {
        "width": grid_size,
        "height": grid_size,
        "num_blocks": num_blocks,
        "start_pos": None,
        "goal_pos": None,
        "edited": False,
        "seed_val": np.random.randint(0, 999999),
    }
    
def edit_config(old_config):
    max_blocks = int(((old_config["width"] - 1) * (old_config["height"] - 1)) / 2)
    
    new_config = dict(old_config)
    
    # Randomly increase number of blocks by 1
    new_number_blocks = old_config["num_blocks"] + np.random.choice([1])
    new_config["num_blocks"] = max(1, min(new_number_blocks, max_blocks))    

    new_config["edited"] = True
    return new_config


# ====================================================
# 2. Simple “level buffer” 
# ====================================================
class LevelBuffer: 
    def __init__(self, max_size=50):
        self.max_size = max_size
        self.data = []  # will store (config_dict, score)

    def add(self, config, score):
        self.data.append((config, score))
        if len(self.data) > self.max_size:
            self.data.sort(key=lambda x: x[1], reverse=True)
            self.data = self.data[: self.max_size]

    def sample_config(self): 
        if len(self.data) == 0:
            return None
        scores = [item[1] for item in self.data]
        total = sum(scores)
        if total <= 1e-9:
            # fallback to uniform
            idx = np.random.randint(len(self.data))
            return self.data[idx][0]
        probs = [s / total for s in scores]
        idx = np.random.choice(len(self.data), p=probs)
        return self.data[idx][0]


# ====================================================
# 3. Utility Functions
# ====================================================
class MinigridFeaturesExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: gym.Space, features_dim: int = 512, normalized_image: bool = False) -> None:
        super().__init__(observation_space, features_dim)
        n_input_channels = observation_space.shape[0]
        self.cnn = nn.Sequential(
            nn.Conv2d(n_input_channels, 16, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(16, 32, (2, 2)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (2, 2)),
            nn.ReLU(),
            nn.Flatten(),
        )

        # Compute shape by doing one forward pass
        with torch.no_grad():
            sample_input = torch.as_tensor(observation_space.sample()[None]).float()
            n_flatten = self.cnn(sample_input).shape[1]

        self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

    def forward(self, observations: torch.Tensor) -> torch.Tensor:
        return self.linear(self.cnn(observations))

policy_kwargs = dict(
    features_extractor_class=MinigridFeaturesExtractor,
    features_extractor_kwargs=dict(features_dim=128),
)


def initialize_ppo(env, learning_rate=1e-4):
    return PPO(
        "CnnPolicy",                    
        env,                            
        verbose=0,                      
        n_steps=256,                    
        batch_size=64,                  
        learning_rate=learning_rate,
        policy_kwargs=policy_kwargs,    
        device=device                   
    )

def print_level_from_config(config):
    env = MyCustomGrid(config, render_mode='rgb_array')
    env.reset()
    full_level_image = env.render()  # This should return an RGB image of the full grid

    plt.figure(figsize=(4, 4))
    plt.imshow(full_level_image)
    plt.title("Level Configuration: " + str(config))
    plt.axis("off")
    plt.show()

def create_vectorized_env(config, n_envs=4):
    env_fns = [lambda: MyCustomGrid(config) for _ in range(n_envs)]
    return make_vec_env(lambda: MyCustomGrid(config), n_envs=n_envs)


# ====================================================
# 4. Parallel regret calculation (NEW)
# ====================================================
def calculate_regret_gae_parallel(
    env_config, 
    model, 
    max_steps=1000, 
    gamma=0.99, 
    lam=0.95, 
    n_envs=8
):
    """
    Roll out n_envs copies of MyCustomGrid(env_config) in parallel,
    compute GAE-based 'regret' for each environment, and return the max.
    """

    # Create vectorized env with n_envs copies
    vec_env = make_vec_env(lambda: MyCustomGrid(env_config), n_envs=n_envs)
    obs_array = vec_env.reset()  # shape: (n_envs, height, width, 3)

    # For each environment, we will store transitions to later compute GAE
    # We'll keep them in lists, one per environment.
    # Alternatively, we can store them in big arrays (n_envs, max_steps), etc.
    rewards_list = [[] for _ in range(n_envs)]
    values_list = [[] for _ in range(n_envs)]
    dones_list = [[] for _ in range(n_envs)]

    for t in range(max_steps):
        # Model’s predict can handle multiple obs in a single forward pass
        actions, _ = model.predict(obs_array, deterministic=True)
        
        # Also compute the values in one batch
        # Convert obs_array to torch tensor
        obs_tensor = torch.as_tensor(np.transpose(obs_array, (0, 3, 1, 2))).float().to(device)
        with torch.no_grad():
            # shape: (n_envs, 1)
            value_t = model.policy.predict_values(obs_tensor).cpu().numpy().flatten()

        # Step all envs in parallel
        next_obs_array, rewards, dones, truncs = vec_env.step(actions)

        # Store the results
        for i in range(n_envs):
            rewards_list[i].append(rewards[i])
            values_list[i].append(value_t[i])
            dones_list[i].append(dones[i] or truncs[i])

        obs_array = next_obs_array

        # If all envs are done or truncated, we can break early
        if all(dones) or all(truncs):
            break

    # We also need the terminal value for each env
    # (0 if done, otherwise model's value at final obs)
    obs_tensor = torch.as_tensor(np.transpose(obs_array, (0, 3, 1, 2))).float().to(device)
    with torch.no_grad():
        final_values = model.policy.predict_values(obs_tensor).cpu().numpy().flatten()

    # Now, compute GAE-based "regret" for each of the n_envs
    regrets = []
    for i in range(n_envs):
        # If the env ended with done or truncated, terminal value = 0
        if dones_list[i][-1]:
            values_list[i].append(0.0)
        else:
            values_list[i].append(final_values[i])

        # Compute delta_t and approximate GAE-like metric
        env_rewards = rewards_list[i]
        env_values = values_list[i]
        env_dones = dones_list[i]

        env_regrets = []
        for t in range(len(env_rewards)):
            delta_t = (
                env_rewards[t] 
                + gamma * env_values[t + 1] * (1 - env_dones[t]) 
                - env_values[t]
            )
            # accumulate discounted error
            discounted_error = (gamma * lam) ** t * delta_t
            env_regrets.append(max(0, discounted_error))

        # The environment's "regret" is the max of its positive GAE deltas
        regrets.append(max(env_regrets) if env_regrets else 0.0)

    # Return the maximum regret across the parallel envs
    return max(regrets) if regrets else 0.0


# ====================================================
# 5. Main ACCEL loop, now using parallel regret
# ====================================================
def main_accel(
    total_iterations, 
    replay_prob, 
    train_steps, 
    level_buffer_size,
    initial_fill_size, 
    grid_size, 
    n_envs, 
    edit_levels, 
    regret_threshold,
    easy_start
):
    # Create a level buffer
    level_buffer = LevelBuffer(max_size=level_buffer_size)
    iteration_regrets = []

    # Use a random config to create a vectorized environment
    dummy_config = random_config(grid_size)
    vectorized_env = create_vectorized_env(dummy_config, n_envs=n_envs)

    # Initialize PPO with vectorized environment
    print("Initializing student model PPO...")
    student_model = initialize_ppo(vectorized_env)

    # Populate buffer with initial levels
    print(f"Populating buffer with {initial_fill_size} initial levels (regret > {regret_threshold})...")
    while len(level_buffer.data) < initial_fill_size:
        if easy_start:
            cfg = random_config(grid_size, num_blocks=2)
        else:
            cfg = random_config(grid_size)

        # *** Use parallel regret calc here (1 env is also valid if you prefer) ***
        regret = calculate_regret_gae_parallel(
            cfg, student_model, 
            max_steps=1000, gamma=0.99, lam=0.95, 
            n_envs=n_envs  # you can tune this
        )

        if regret < regret_threshold:
            continue

        level_buffer.add(cfg, regret)

    # Main ACCEL loop
    iteration, skipped = 0, 0
    print("\nMain training loop...")
    while iteration < total_iterations + skipped:
        print(f"\n=== ITERATION {iteration + 1}/{total_iterations + skipped} SKIPPED: {skipped} ===")

        iteration += 1

        # Decide whether to replay or generate a new level
        use_replay = np.random.rand() < replay_prob

        if not use_replay or len(level_buffer.data) == 0:
            cfg = random_config(grid_size)
            print("Generated new random level:", cfg)
        else:
            cfg = level_buffer.sample_config()
            print("Sampled level from buffer:", cfg)

        # Update the vectorized environment with the selected config
        vectorized_env = create_vectorized_env(cfg, n_envs=n_envs)
        student_model.set_env(vectorized_env)

        # Train the student model
        student_model.learn(total_timesteps=train_steps)

        if not use_replay or not edit_levels:
            regret = calculate_regret_gae_parallel(
                cfg, student_model, 
                max_steps=1000, gamma=0.99, lam=0.95,
                n_envs=n_envs  # parallel
            )
        else:
            # Edit the level and calculate regret
            cfg = edit_config(cfg)
            print("Edited level to:", cfg)
            regret = calculate_regret_gae_parallel(
                cfg, student_model, 
                max_steps=1000, gamma=0.99, lam=0.95,
                n_envs=n_envs
            )

        if regret <= regret_threshold:
            print(f"Regret={regret:.5f} <= threshold={regret_threshold}. Skipping...")
            skipped += 1
            continue

        print(f"Regret for current level: {regret}")
        level_buffer.add(cfg, regret)
        iteration_regrets.append(regret)

    # Plot the progress
    plt.figure(figsize=(8, 4))
    plt.plot(iteration_regrets, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Regret")
    plt.title("Regret Progress during ACCEL Training")
    plt.grid(True)
    plt.show()

    print("\nDone. Final buffer size:", len(level_buffer.data))
    print("Top-5 hardest levels (config, regret):")
    level_buffer.data.sort(key=lambda x: x[1], reverse=True)
    for i, (cfg, sc) in enumerate(level_buffer.data[:5]):
        print(f"{i + 1}. regret={sc:.5f}, config={cfg}")

    print("Top-5 easiest levels (config, regret):")
    level_buffer.data.sort(key=lambda x: x[1])
    for i, (cfg, sc) in enumerate(level_buffer.data[:5]):
        print(f"{i + 1}. regret={sc:.5f}, config={cfg}")

    return student_model


# ============================
# Example of how you might run:
# ============================
if __name__ == "__main__":
    trained_model = main_accel(
        total_iterations=5,
        replay_prob=0.5,
        train_steps=5000,
        level_buffer_size=10,
        initial_fill_size=5,
        grid_size=8,
        n_envs=4,             # we run 4 environments in parallel
        edit_levels=True,
        regret_threshold=0.1,
        easy_start=False
    )
